# my sql 테이블 생성 및 IO

In [3]:
import json
import pymysql

In [4]:
with open('data/mysql.json', 'r') as file: 
    config_str = file.read() 
    config = json.loads(config_str)
    
conn = pymysql.connect(
    host = config['host'],
    user = config['user'],
    password=config['password'],
    database=config['database'],
    port=config['port']
) 

## 테이블 생성

In [6]:
cur = conn.cursor()
sql_c_t ='''
    create table if not EXISTS users (
        uid varchar(20) not null primary KEY,
        pwd char(44) not null, uname varchar(20) not NULL,
        email varchar(40),
        reg_date datetime default CURRENT_TIMESTAMP,
        is_deleted int default 0
    );
'''
cur.execute(sql_c_t)


0

## 데이터 추가

In [ ]:
sql_insert='''
    INSERT INTO users(uid, uname, pwd) VALUES('admin', '관리자', '1234');
'''
cur.execute(sql_insert)

In [9]:
sql_select='''
    select * from users
    '''
cur.execute(sql_select)
row = cur.fetchone()

In [10]:
row

('admin', '1234', '관리자', None, datetime.datetime(2021, 8, 25, 13, 28, 28), 0)

# 위에 것들은 캐쉬메모리에 써져있다
# 이상태에서 db 보면 추가 안되어 있다
# 확실히 쓰게 하는명령어 conn.commit()

In [11]:
conn.commit() # 확실히 db 로 보내는 명령어 conn.commit()

### 여러건의 데이터 추가

In [13]:
sql_multi_insert="""
    INSERT INTO users(uid, pwd, uname) VALUES
    ('eskim', '1234', '김은숙'), 
    ('wjlee', '1234', '이우정');
"""
cur.execute(sql_multi_insert)
conn.commit()


In [14]:
sql_insert_ph = """
INSERT INTO users(uid, pwd, uname) VALUES(%s, '1234', %s);
"""
uid, uname = 'djy', '대조영'
cur.execute(sql_insert_ph,(uid, uname)) #튜플로 넣는다
conn.commit()

### 여러건의 데이터를 placeholder를 이용해서 insert

In [17]:
users = (('gdhong', '홍길동'), ('jbpark', '박재범'))

sql_insert_ph = """
    insert into users(uid, pwd, uname) values(%s, '1234', %s);
"""


In [19]:
cur.executemany(sql_insert_ph, users)
conn.commit()

In [21]:
#강사 추천
users = (('gdhong2', '홍길동'), ('jbpark2', '박재범'))
for user in users:
    cur.execute(sql_insert_ph, user)
conn.commit()


## 매무리

In [22]:
cur.close()
conn.close()